|Faculdade |Curso|Disciplina |Professor|Ano|
| :- | :- | :- | :- | :- |
|Estácio de São Paulo|ADS|Tópicos de Big Data em Python|Prof. Fabio Oliveira|2025|

# Análise de Dados

### 📘 Dicionário de Dados - Base de Filmes

Este dicionário descreve os campos presentes na base de dados de filmes utilizada para análise.

| Nome do Campo       | Tipo de Dado | Descrição                                               | Valores Possíveis / Exemplos             |
|---------------------|--------------|----------------------------------------------------------|------------------------------------------|
| `cliente_id`        | Inteiro      | Identificador único do cliente                          | 1001, 1002, 1003                          |
| `nome`              | Texto        | Nome completo do cliente                                | "Maria Silva", "João Souza"              |
| `data_nascimento`   | Data         | Data de nascimento do cliente                           | "1985-07-23"                              |
| `sexo`              | Texto        | Gênero do cliente                                        | "Feminino", "Masculino", "Outro"         |
| `estado`            | Texto        | Estado de residência                                     | "SP", "RJ", "MG", "RS"                   |
| `renda_mensal`      | Decimal      | Renda mensal declarada                                   | 2500.00, 7800.50                          |
| `data_cadastro`     | Data         | Data de cadastro na plataforma                          | "2021-03-15"                              |
| `ativo`             | Booleano     | Indica se o cliente está ativo                          | `True`, `False`                          |
| `categoria_cliente` | Texto        | Segmentação do cliente com base em perfil de consumo     | "Premium", "Regular", "Básico"           |
| `score_credito`     | Inteiro      | Pontuação de crédito atribuída                          | 300–850                                  |


